In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler,normalize
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split,KFold,cross_val_score,RepeatedKFold
from sklearn.metrics import r2_score,accuracy_score,mean_absolute_error,mean_squared_error

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [15]:
def datapreprocess(df):
    df['day']=df['datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M').day)
    df['month']=df['datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M').month)
    df['year']=df['datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M').year)
    df['hour']=df['datetime'].apply(lambda x:datetime.strptime(x,'%m/%d/%Y %H:%M').hour)
    df.drop('datetime',axis=1,inplace=True)
    #for col in ['season', 'holiday', 'workingday', 'month','hour','weather','day']:
        #df[col] = df[col].astype('category')
    weather_map={'Clear + Few clouds':'CF','Light Snow, Light Rain':'LSLR','Mist + Cloudy':'MC','Heavy Rain + Thunderstorm':'HRT'}
    df['weather'] = df['weather'].apply(lambda x : weather_map.get(x.strip()))
    

In [16]:
def catgoricalencoding(df):
    cols_for_label_encoding=['season','weather','year','month','hour','day']
    dataset = pd.get_dummies(columns=cols_for_label_encoding, data=df)
    return dataset

In [4]:
def get_outlier(df):
    features = df.columns
    outliers  = []
    for i, feature in enumerate(features):
        if ( (df[feature].dtype == 'float64')):
            # Calculate Q1 (25th percentile of the data) for the given feature
            Q1 = np.percentile(df[feature], 25)
            # Calculate Q3 (75th percentile of the data) for the given feature
            Q3 = np.percentile(df[feature], 75)
            # Use the interquartile range to calculate an outlier step
            step = 1.5 * (Q3 - Q1)
            feature_outliers = df[~((df[feature] >= Q1 - step) & (df[feature] <= Q3 + step))]
            outliers.extend(list(feature_outliers.index.values))
            print('Feature Name: {}, No. of outliers: {}\n'.format(feature, len(feature_outliers.index)))
    
    multi_feature_outliers = (Counter(outliers) - Counter(set(outliers))).keys()
    #print(outliers)
    return outliers

In [29]:
train=pd.read_csv('train.csv')
datapreprocess(train)
train_laebl=pd.read_csv('train_label.csv',header=None)
train_laebl.rename(columns={0:'Total_bookings'},inplace=True)
train['Total_bookings']=train_laebl['Total_bookings']


In [30]:
from collections import Counter
multi_feature_outliers = get_outlier(train)

Feature Name: temp, No. of outliers: 0

Feature Name: atemp, No. of outliers: 0

Feature Name: windspeed, No. of outliers: 182



In [31]:
train=train.drop(train.index[list(multi_feature_outliers)]).reset_index(drop=True)

In [32]:
dataset=catgoricalencoding(train)

Y=dataset['Total_bookings']

dataset.drop(['atemp','weather_HRT','Total_bookings'],axis=1,inplace=True)

In [33]:
def scaleData(dataset):
    standardScaler = StandardScaler()
    scaled_data = standardScaler.fit_transform(dataset[['humidity','temp','windspeed']])
    scaled_data_df=pd.DataFrame(scaled_data,columns=['humidity','temp','windspeed'])
    dataset[['humidity','temp','windspeed']] = scaled_data_df[['humidity','temp','windspeed']]
    return dataset
dataseta=scaleData(dataset)

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [34]:
X_train, X_test,y_train, y_test = train_test_split(dataset,Y,test_size=0.3,random_state=2)

In [35]:
xgbRegressor = XGBRegressor()

In [36]:
 
xgbRegressor.fit(X_train,y_train)

D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:38:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [37]:
xgbRegressor.score(X_train,y_train)

0.8117821214840522

In [38]:
y_predict=xgbRegressor.predict(X_test)

In [39]:
r2_score(y_test,y_predict) , mean_absolute_error(y_test,y_predict)

(0.8128488814874986, 56.65761078047137)

CHECK ON TEST DATA

In [26]:
test=pd.read_csv('test.csv')
test_label=pd.read_csv('test_label.csv',header=None)
datapreprocess(test)
test_data=catgoricalencoding(test)
test_data.drop(['atemp'],axis=1,inplace=True)
test_data=scaleData(test_data)

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [27]:
bookng_predict_test=xgbRegressor.predict(test_data)

In [28]:
r2_score(test_label,bookng_predict_test) , mean_absolute_error(test_label,bookng_predict_test)

(0.8028050895990515, 58.12419634507379)

In [82]:
scores=[]
learning_rate=[]
n_estimator=[]
perforMetrics=pd.DataFrame(columns=['learningrate','nestimator','score','r2score','mse'])

XGBoost Hyper parameter tuning

In [138]:
learningRate=[0.2]
nestimator=2000
for lr in learningRate:
    xgb2=XGBRegressor(n_estimators= nestimator,objective='reg:squarederror',colsample_bytree=1,learning_rate=lr)
    xgb2.fit(X_train,y_train)
    score=xgb2.score(X_train,y_train)
    print('score',score)
    y_predict=xgb2.predict(X_test)
    r2score,mse=r2_score(y_test,y_predict) , np.sqrt(mean_squared_error(y_test,y_predict))
    perforMetrics = perforMetrics.append({'learningrate':lr,'nestimator':nestimator,'score':score,'r2score':r2score,'mse':mse},ignore_index=True)


D:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


score 0.9838035413662984


In [139]:
perforMetrics

,learningrate,nestimator,score,r2score,mse
0,0.1,100.0,0.811782,0.812849,78.833363
1,0.1,300.0,0.902730,0.890433,60.318859
2,0.1,500.0,0.932705,0.915457,52.984842
3,0.1,1000.0,0.951984,0.924314,50.132894
4,0.1,1500.0,0.963112,0.929432,48.407903
5,0.1,2000.0,0.969703,0.931288,47.767379
6,0.1,5000.0,0.985929,0.931683,47.629837
7,0.1,10000.0,0.993676,0.927911,48.927106
8,0.1,13000.0,0.995609,0.926478,49.410709
9,0.2,1000.0,0.971611,0.932149,47.466979
